# Urban Heat Island Analysis – Bologna

This notebook performs a comprehensive geospatial analysis to identify and classify Urban Heat Island (UHI) effects in the city of Bologna, Italy, using satellite data and derived indices.

## Process Overview

1. **Load AOI (Bologna)** from precise administrative boundary (GeoJSON)
2. **Retrieve satellite datasets**:
   - **Landsat 8/9 SR** for LST, NDVI, and Albedo (summer 2024)
   - **MODIS LST** for day/night surface temperature (summer 2024)
3. **Calculate indices**:
   - **LST (Land Surface Temperature)**: from ST_B10 (Landsat)
   - **Z-score of LST**: statistical anomaly per pixel
   - **NDVI**: vegetation index from SR_B4 and SR_B5
   - **Albedo**: surface reflectance from SR_B2, B4, B5, B6, B7 using Liang (2001) coefficients
   - **Heat-Vegetation Index**: LST norm – NDVI norm
   - **Heat Retention Index**: LST norm – Albedo norm
   - **MODIS ΔLST (Day – Night)** and its classification
4. **Classify and export outputs**:
   - Classify Z-score, Albedo, ΔLST
   - Export all outputs as raster and GeoJSON (dissolved)
5. **Visualize layers** with dynamic legend and map interactivity

In [1]:
import ee
import os
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
from rasterio import features
from shapely.geometry import shape
from shapely.ops import unary_union
import json
from shapely.geometry import MultiPolygon
from rasterio.features import shapes
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union, polygonize
import leafmap
from ipywidgets import Output

In [2]:
DATADIR="data"
os.makedirs(DATADIR, exist_ok=True)

In [3]:
def kelvin_to_celsius(img):
    return img.select("ST_B10").multiply(0.00341802).add(149.0).subtract(273.15).rename("LST")

In [4]:
def compute_ndvi(img):
    sr = img.select(["SR_B5", "SR_B4"]).multiply(0.0000275).add(-0.2)
    ndvi = sr.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    return img.addBands(ndvi)

In [5]:
def compute_albedo(img):
    sr = img.multiply(0.0000275).add(-0.2)
    b2 = sr.select("SR_B2")
    b4 = sr.select("SR_B4")
    b5 = sr.select("SR_B5")
    b6 = sr.select("SR_B6")
    b7 = sr.select("SR_B7")
    albedo = (b2.multiply(0.356)
                .add(b4.multiply(0.130))
                .add(b5.multiply(0.373))
                .add(b6.multiply(0.085))
                .add(b7.multiply(0.072))
                .subtract(0.0018))
    return img.addBands(albedo.rename("Albedo"))

In [6]:
def raster_to_dissolved_geojson(raster_path, geojson_path):
    with rasterio.open(raster_path) as src:
        image = src.read(1)
        mask = image != src.nodata
        results = (
            {'properties': {'value': int(v)}, 'geometry': shape(s)}
            for s, v in shapes(image, mask=mask, transform=src.transform)
        )
        gdf = gpd.GeoDataFrame.from_features(list(results), crs=src.crs)
        gdf = gdf.dissolve(by='value').reset_index()
        gdf.to_file(geojson_path, driver='GeoJSON')

## Load AOI 
Load precise municipal boundary of Bologna from local GeoJSON<br/>
Required for spatial clipping of all datasets


In [7]:
collectdata = False
gdf = gpd.read_file("bologna.geojson").to_crs("EPSG:4326")



# Data Preparation
## Extract data from Google Earth Engine


In [8]:
if not os.path.exists(DATADIR + os.sep + 'lst_mean.tif'):
    collectdata = True

In [9]:
if collectdata:
    ee.Authenticate()
    ee.Initialize(project='ndvi-423516')    
    bologna_fc = geemap.geopandas_to_ee(gdf)
    bologna_geom = bologna_fc.geometry()

## Retrieve MODIS LST (Day/Night) from MOD11A1 – Summer 2024 
MODIS Terra (MOD11A1) provides daily 1 km LST for day and night<br/>
We'll calculate the difference between day and night to assess heat retention

In [10]:
    modis = ee.ImageCollection("MODIS/061/MOD11A1") \
        .filterBounds(bologna_geom) \
        .filterDate("2024-06-01", "2024-08-31")


EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

### Classify ΔLST Day-Night 
Classify difference into three heat release classes:
- Class 1: fast cooling (>10°C)
- Class 2: moderate (5–10°C)
- Class 3: low cooling (<5°C), i.e. high heat retention

In [ ]:

    lst_day_modis = modis.select("LST_Day_1km").mean().multiply(0.02).subtract(273.15).rename("LST_Day")
    lst_night_modis = modis.select("LST_Night_1km").mean().multiply(0.02).subtract(273.15).rename("LST_Night")
    delta_lst = lst_day_modis.subtract(lst_night_modis).rename("DeltaLST")
    delta_lst_class = delta_lst.expression("(d < 5) ? 3 : (d < 10) ? 2 : 1", {"d": delta_lst}).rename("DeltaLSTClass")

## Retrieve Landsat 8/9 Collection 2 L2 imagery (summer 2024) 
- Purpose: estimate Land Surface Temperature (LST) from band ST_B10
- LST will be used as baseline to analyze surface thermal variability

In [ ]:
    collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterBounds(bologna_geom)
        .filterDate("2024-06-01", "2024-08-31")
        .filter(ee.Filter.lt("CLOUD_COVER", 10))
        .map(kelvin_to_celsius))



### Calculate LST Z-score
- Identify statistical anomalies in LST relative to the city's average
- Z = (pixel - mean) / std_dev

In [ ]:
    lst_mean = collection.mean().clip(bologna_geom)
    mean_dict = lst_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=bologna_geom, scale=100, maxPixels=1e13)
    std_dict = lst_mean.reduceRegion(reducer=ee.Reducer.stdDev(), geometry=bologna_geom, scale=100, maxPixels=1e13)
    mean_val = ee.Number(mean_dict.get("LST"))
    std_val = ee.Number(std_dict.get("LST"))
    z_score = lst_mean.subtract(mean_val).divide(std_val).rename("Z")


### Classify LST Z-score into 10 classes (5 below, 5 above average) 
This helps in highlighting both cool and hot zones relative to the norm

In [ ]:
    classified = z_score.expression("(z <= -2.0) ? 1 : (z <= -1.5) ? 2 : (z <= -1.0) ? 3 : (z <= -0.5) ? 4 : " +
                                        "(z < 0) ? 5 : (z < 0.5) ? 6 : (z < 1.0) ? 7 : (z < 1.5) ? 8 : (z < 2.0) ? 9 : 10", {"z": z_score}).rename("Z_class")

##  Compute NDVI (Normalized Difference Vegetation Index) 
NDVI = (NIR - Red) / (NIR + Red)<br/>
Derived from Surface Reflectance bands B5 (NIR) and B4 (Red)<br/>
NDVI is used to identify vegetated vs built-up areas

In [ ]:

    ndvi_collection = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        .filterBounds(bologna_geom)
        .filterDate("2024-06-01", "2024-08-31")
        .filter(ee.Filter.lt("CLOUD_COVER", 10))
        .map(compute_ndvi))

    ndvi_mean = ndvi_collection.select("NDVI").mean().clip(bologna_geom)

## Compute Albedo using Liang (2001) method
Albedo is calculated using a weighted sum of Surface Reflectance bands<br>
Equation adapted for Landsat 8: B2, B4, B5, B6, B7<br>
Result: surface reflectance from 0 (fully absorbent) to 1 (fully reflective)


In [ ]:

    albedo_collection = ndvi_collection.map(compute_albedo)
    albedo_mean = albedo_collection.select("Albedo").mean().clip(bologna_geom)

### Normalize LST, NDVI, and Albedo 
- Normalize LST (30–50°C), NDVI (0–0.8), Albedo (0.05–0.35) to [0, 1] range
- Needed for consistent index construction

In [ ]:
    lst_norm = lst_mean.unitScale(30, 50).rename("LST_norm")
    ndvi_norm = ndvi_mean.unitScale(0, 0.8).rename("NDVI_norm")
    albedo_norm = albedo_mean.unitScale(0.05, 0.35).rename("Albedo_norm")


## Compute Heat-Vegetation Index 
High value = hot and poorly vegetated → risk of UHI


In [ ]:
    heat_veg_index = lst_norm.subtract(ndvi_norm).rename("HeatVegIndex")
    heat_retention_index = lst_norm.subtract(albedo_norm).rename("HeatRetentionIndex")

## Compute Heat Retention Index
High value = hot and low albedo → likely to retain heat during night

In [ ]:
heat_retention_index = lst_norm.subtract(albedo_norm).rename("HeatRetentionIndex")

### Export Raster Outputs as GeoTIFF (clipped to Bologna) 
Save all indices and classified rasters as GeoTIFFs for further analysis

In [ ]:
if not os.path.exists(DATADIR + os.sep + 'lst_mean.tif'):
    geemap.ee_export_image(lst_mean.clip(bologna_geom), filename=DATADIR + os.sep + 'lst_mean.tif', scale=100, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'lst_mean.tif', DATADIR + os.sep + 'lst_mean.geojson')
    
if not os.path.exists(DATADIR + os.sep + 'z_score.tif'):
    geemap.ee_export_image(z_score.clip(bologna_geom), filename=DATADIR + os.sep + 'z_score.tif', scale=100, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'z_score.tif', DATADIR + os.sep + 'z_score.geojson')
    
if not os.path.exists(DATADIR + os.sep + 'z_score_class.tif'):
    geemap.ee_export_image(classified.clip(bologna_geom), filename=DATADIR + os.sep + 'z_score_class.tif', scale=100, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'z_score_class.tif', DATADIR + os.sep + 'z_score_class.geojson')
    
if not os.path.exists(DATADIR + os.sep + 'ndvi_mean.tif'):
    geemap.ee_export_image(ndvi_mean.clip(bologna_geom), filename=DATADIR + os.sep + 'ndvi_mean.tif', scale=30, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'ndvi_mean.tif', DATADIR + os.sep + 'ndvi_mean.geojson')
    
if not os.path.exists(DATADIR + os.sep + 'albedo_mean.tif'):
    geemap.ee_export_image(albedo_mean.clip(bologna_geom), filename=DATADIR + os.sep + 'albedo_mean.tif', scale=30, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'albedo_mean.tif', DATADIR + os.sep + 'albedo_mean.geojson')

if not os.path.exists(DATADIR + os.sep + 'heat_veg_index.tif'):
    geemap.ee_export_image(heat_veg_index.clip(bologna_geom), filename=DATADIR + os.sep + 'heat_veg_index.tif', scale=100, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'heat_veg_index.tif', DATADIR + os.sep + 'heat_veg_index.geojson')

if not os.path.exists(DATADIR + os.sep + 'heat_retention_index.tif'):
    geemap.ee_export_image(heat_retention_index.clip(bologna_geom), filename=DATADIR + os.sep + 'heat_retention_index.tif', scale=100, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'heat_retention_index.tif', DATADIR + os.sep + 'heat_retention_index.geojson')

if not os.path.exists(DATADIR + os.sep + 'lst_day_modis.tif'):
    geemap.ee_export_image(lst_day_modis.clip(bologna_geom), filename=DATADIR + os.sep + 'lst_day_modis.tif', scale=1000, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'lst_day_modis.tif', DATADIR + os.sep + 'lst_day_modis.geojson')

if not os.path.exists(DATADIR + os.sep + 'lst_night_modis.tif'):
    geemap.ee_export_image(lst_night_modis.clip(bologna_geom), filename=DATADIR + os.sep + 'lst_night_modis.tif', scale=1000, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'lst_night_modis.tif', DATADIR + os.sep + 'lst_night_modis.geojson')

if not os.path.exists(DATADIR + os.sep + 'delta_lst.tif'):
    geemap.ee_export_image(delta_lst.clip(bologna_geom), filename=DATADIR + os.sep + 'delta_lst.tif', scale=1000, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'delta_lst.tif', DATADIR + os.sep + 'delta_lst.geojson')

if not os.path.exists(DATADIR + os.sep + 'delta_lst_class.tif'):
    geemap.ee_export_image(delta_lst_class.clip(bologna_geom), filename=DATADIR + os.sep + 'delta_lst_class.tif', scale=1000, region=bologna_geom)
    raster_to_dissolved_geojson(DATADIR + os.sep + 'delta_lst_class.tif', DATADIR + os.sep + 'delta_lst_class.geojson')

    


Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/lst_mean.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/z_score.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/z_score_class.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/ndvi_mean.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/albedo_mean.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/heat_veg_index.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/heat_retention_index.tif
Generating URL ...
Please wait ...
Data downloaded to /home/napo/Documents/development/talea/uhi/data/lst_day_modis.tif
Generating URL ...
Please wait ...
Data downloade

## Visualize Layers on Interactive Map
- Display the key indices and classified maps on an interactive map
- Each layer includes an appropriate color palette for interpretation